In [1]:
import pandas as pd
import json
import os.path

### Notes

In [2]:
notes = pd.read_csv("data/mimic_notes.csv")

/users/k1810895/.conda/envs/py3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# notes.head()

In [4]:
notes[notes['row_id'].isna()==True]

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text,gender,dob,dod,dod_hosp,dod_ssn,expire_flag,etl_date


In [5]:
notes.shape

(2083180, 18)

### Scispacy annotations

In [33]:
i = 0

In [ ]:
# scispacy
batch = 1001
ds = set()
for i in range(batch):
    try:
        d = json.load(open('data/scispacy/batch' + str(i) +'.json',"r"))
    except Exception as e:
        print(i)
        continue
    for did in d.keys():
        ds.add(int(did))

In [37]:
miss_rows = (set(notes['row_id']) - (ds))

In [38]:
len(miss_rows)

0

In [39]:
len(ds)

2083180

### Test scispacy on missing documents

In [20]:
import scispacy
import spacy
import pandas as pd
import sys
from datetime import datetime

In [21]:
from scispacy.linking import EntityLinker

nlp = spacy.load("en_core_sci_scibert")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "name": "umls"})

/users/k1810895/.conda/envs/py3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/users/k1810895/.conda/envs/py3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [22]:
nts = notes[notes['row_id'].isin(list(miss_rows)) == True]
print((nts.shape))

(3324, 18)


In [76]:
linker = nlp.get_pipe("scispacy_linker")

In [77]:
text = '''
	
NEURO;  PT ALERT & ORIENTED X 3, FOLLOWS SIMPLE COMMANDS, LIFTS AND HOLDS LEFT ARM, RT ARM WEAKER AND PT'S ARM FALLS ON MATTRESS WHEN RAISED, WIGGLES TOES BUT NOT ABLE TO RAISE LEGS OFF BED, LEFT EYELID DROOP NOTED [**Name (NI) **], PT TAKEN TO MRI ON EVES, ? HEMATOMA AT L4, ORTHO TEAM WILL EVALUATE, MEDIC WITH PERCOCET 1 TAB WITH FAIRLY GOOD EFFECT, PT SLEPT FOR SEVERAL HRS THROUGHOUT NOC,

CARDIOVASCULAR;  HR 90'S TO 100'S, A FIB, SYS BP 150'S,-140'S,

RESPIR;  LUNGS CLEAR THIS EVE, TAKING DEEP BREATHS WITH ENCOURAGEMENT, ON R/A WITH 02 SAT 98%

ABDOMEN;  SOFT, BS PRESENT, TUBE FEEDS OFF AT MIDNOC FOR RPT SPEECH AND SWALLOW TEST, MAIN IV INCREASED TO 80CC/HR AFTER TUBE FEEDS OFF

SKIN;  WARM AND DRY, TEMP 100 AX-99.1PO, MEDIC WITH TYLENOL, BACK DSG D/I, STERI-STRIPS LOWER LUMBAR AREA, INTACT

HEMATOLOGIC;  AM HCT 27, H.O. INFORMED, TX DEFERRED, K 4.0, ION CA 1.12-NO REPLETION ORDERED


'''

In [95]:
doc = nlp(text)
for ent in doc.ents:
    for umls_ent in ent._.kb_ents[:1]:
        print(ent.start_char, ent.end_char, ent.text, umls_ent[0], umls_ent[1])
        entity = (linker.kb.cui_to_entity[umls_ent[0]])
#         print(detail)
        print(entity.canonical_name, ','.join(entity.types))
#         print(ent)

14 19 ALERT C0239110 0.9999999403953552
Consciousness clear T033
22 34 ORIENTED X 3 C1704322 0.8068607449531555
Orientation (spatial) T082
36 43 FOLLOWS C0332283 0.9999998807907104
Followed by T079
44 50 SIMPLE C0205352 1.0
Simple T080
51 59 COMMANDS C3639196 1.0
CDISC ADAS-Cog - Commands Summary Score T170
61 66 LIFTS C0181620 1.0
Hoist device T074
71 76 HOLDS C0009443 0.7643272280693054
Common Cold T047
77 85 LEFT ARM C0230347 1.0
Left upper arm structure T023
137 143 RAISED C0442818 1.0
Raised T080
162 170 NOT ABLE C4048706 0.7585674524307251
Unable to Do T033
174 184 RAISE LEGS C1140621 0.7350113987922668
Leg T023
185 188 OFF C1518543 1.0
Off T080
189 192 BED C0004916 1.0
Beds T073
194 217 LEFT EYELID DROOP NOTED C0005745 0.8324488997459412
Blepharoptosis T047
305 310 MEDIC C0199168 0.7410444021224976
Medical service T058
327 330 TAB C0039225 1.0
Tablet Dosage Form T122
336 342 FAIRLY C4086298 0.9045276641845703
Fairly Sure T170
343 347 GOOD C0205170 1.0
Good T080
348 354 EFFECT C1

In [39]:
'1286796' in nts['row_id']

False

In [60]:
dlist = {}
for row_id, subject_id, text in nts[['row_id', 'subject_id', 'text']].values:
#             print('Document ID: ', row_id)
    try:
        doc = nlp(text)
    except Exception as e:
        if 'RuntimeError' in str(e):
            print(row_id, e)
            continue
    anns = {}
    for ent in doc.ents:
        for umls_ent in ent._.kb_ents[:1]:
            
            dlist.append([row_id, subject_id, ent.start_char, ent.end_char, ent.text.strip(), umls_ent[0], umls_ent[1]])
data = pd.DataFrame(dlist, columns=['row_id', 'subject_id', 'start_char', 'end_char', 'text', 'cui', 'score'])    
data['ann_id'] = data['row_id'].astype(str) + data["start_char"].astype(str)


data.to_csv('data/scispacy/batch' + '1001' +'.csv', index=None)

In [68]:
d = pd.read_csv('data/scispacy/batch' + '1001' +'.csv')

In [69]:
len(set(d['row_id']))

1907

In [70]:
# d['row_id']

In [71]:
# len(miss_rows - set(int(i) for i in d['row_id']))

In [72]:
for i in d['row_id']:
    print(i)
    print(int(i))

677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796
677796

681796
681796
681796
681796
681796
681796
681796
681796
681796
681796
681796
681796
681796
681796
681796
681796
681796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796
690796

691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
691796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796
703796

707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
707796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796
719796

730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
730796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796
727796

...

1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1148796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1140796
1171796
1171796
1171796
1171796


1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1189796
1178796
1178796
1178796
1178796
1178796


1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1215796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796
1234796


1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1284796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796
1288796


ValueError: invalid literal for int() with base 10: 'Care'

In [46]:
len(miss_rows )

3324

## Medcat

In [6]:
# medcat
batch = 1000
ds = set()
for i in range(batch):
    try:
        d = json.load(open('data/medcat/batch' + str(i) +'.json',"r"))
    except Exception as e:
        print(i)
        continue
    for did in d.keys():
        ds.add(int(did))

In [15]:
list(ds)[:3]

[1, 2, 3]

In [14]:
len(ds)

1852549

In [16]:
1 in set(notes['row_id'])

True

In [7]:
miss_rows = (set(notes['row_id']) - (ds))

In [8]:
len(miss_rows)

230631

In [10]:
notes['row_id'].tolist()[:3]

[673887, 673893, 673895]

In [9]:
list(miss_rows)[:3]

[1048581, 1048596, 524309]

In [31]:
# list(miss_rows)

In [13]:
notes.shape

(2083180, 18)

In [17]:
2083180 - 1852549

230631

In [18]:
230631/2083180

0.11071102833168521

#### A large number of documents missing. One possible reason is: the annotations cannot be saved if multiple nodes are used. 